In [8]:
from agno.agent import Agent
from agno.team import Team
from agno.models.openai import OpenAIChat

from sentence_transformers import SentenceTransformer
import faiss
import json
import httpx
import numpy as np

# ✅ Load assets
model = SentenceTransformer("all-MiniLM-L6-v2")
index = faiss.read_index(r"D:\NLPInsuranceProject\NLPINSURANCE-FINTECHPROJ\faiss_policy.index")
with open(r"D:\NLPInsuranceProject\NLPINSURANCE-FINTECHPROJ\policy_metadata.json", "r", encoding="utf-8") as f:
    metadata_docs = json.load(f)

OPENROUTER_API_KEY = "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"
MODEL_NAME = "meta-llama/llama-3-70b-instruct:free"
K = 3

# ✅ Policy Recommender Agent Callable
class PolicyRecommender:
    def __call__(self, query: str) -> dict:
        query_vector = model.encode([query], convert_to_numpy=True)
        D, I = index.search(query_vector, K)
        top_docs = [metadata_docs[i] for i in I[0]]

        context = "\n".join([f"- {doc}" for doc in top_docs])

        prompt = f"""
## 🧾 User Query:
> **"{query}"**

## 📘 Top {K} Matching Policy Descriptions:
{context}

## 🎯 Task:
As an expert assistant, recommend **one** health insurance policy from the list above that best fits the user’s query. Explain your reasoning in 2–4 bullet points using simple language. Keep the response short, clear, and in **Markdown format**.
"""

        headers = {
            "Authorization": f"Bearer {OPENROUTER_API_KEY}",
            "Content-Type": "application/json"
        }

        payload = {
            "model": MODEL_NAME,
            "messages": [
                {"role": "system", "content": "You are a helpful assistant trained in Indian health insurance policy recommendations. Return replies in Markdown with emojis."},
                {"role": "user", "content": prompt}
            ],
            "temperature": 0.4,
            "max_tokens": 512
        }

        response = httpx.post("https://openrouter.ai/api/v1/chat/completions", headers=headers, json=payload)
        response.raise_for_status()
        answer = response.json()["choices"][0]["message"]["content"]

        return {
            "recommendation": answer,
            "top_docs": top_docs
        }

# ✅ Policy Justifier Agent Callable
class PolicyJustifier:
    def __call__(self, input_data: dict) -> str:
        rec = input_data["recommendation"]
        justification_prompt = f"""
## ✅ Selected Policy Recommendation:
{rec}

## 🧠 Task:
Based on this, explain why this policy was chosen over the others. 
Consider differences in premium, OPD cover, cashless network, and brand.
Be clear, add 2–3 key comparison points in Markdown bullets.
"""

        headers = {
            "Authorization": f"Bearer {OPENROUTER_API_KEY}",
            "Content-Type": "application/json"
        }

        payload = {
            "model": MODEL_NAME,
            "messages": [
                {"role": "system", "content": "You are an insurance policy expert who justifies model decisions."},
                {"role": "user", "content": justification_prompt}
            ],
            "temperature": 0.3,
            "max_tokens": 256
        }

        response = httpx.post("https://openrouter.ai/api/v1/chat/completions", headers=headers, json=payload)
        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"]

# ✅ Instantiate Agents (no args in constructor, set via attributes)
policy_recommender_agent = Agent()
policy_recommender_agent.model = PolicyRecommender()
policy_recommender_agent.description = "Retrieves policies and asks LLM to choose the best one"
policy_recommender_agent.markdown = True

justifier_agent = Agent()
justifier_agent.model = PolicyJustifier()
justifier_agent.description = "Explains why the selected policy is the best"
justifier_agent.markdown = True

# ✅ Define Team
recommendation_team = Team()
recommendation_team.agents = [policy_recommender_agent, justifier_agent]
recommendation_team.description = "Team that retrieves and justifies the best policy recommendation for user queries"

# ✅ Run the team
def run_recommendation(query: str):
    print(f"📝 User Query: {query}")
    result = recommendation_team.run(query)
    print("\n🎯 Final Recommendation:\n")
    print(result)

# ✅ Try it
if __name__ == "__main__":
    run_recommendation("I want a budget plan that includes OPD and covers pre-existing conditions.")


ImportError: cannot import name 'nn' from partially initialized module 'torch' (most likely due to a circular import) (d:\anaconda3\Lib\site-packages\torch\__init__.py)